In [ ]:
# Conectar ao banco de dados PostgreSQL usando get_connection do conn.py
import sys
import os

# Adiciona o diretório raiz do projeto ao sys.path para importar conn.py
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, '../../../../..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from conn import get_connection

# Cria e retorna uma nova conexão
conn = get_connection()
cursor = conn.cursor()

# Testa a conexão
try:
    cursor.execute('SELECT version();')
    result = cursor.fetchone()
    print('Conexão bem-sucedida! Versão do banco:', result[0])
except Exception as e:
    print('Erro ao conectar ou executar consulta:', e)
finally:
    cursor.close()
    conn.close()

NameError: name '__file__' is not defined